#### Import the relevant libraries

In [1]:
from PIL import Image
from PIL import ImageFilter
import numpy as np

#### Read and convert to greyscale

Open the image and convert it to greyscale since we are asked to convolve on greyscale images

In [2]:
im = Image.open('test-images/music1.png').convert("L")
pixels = np.array(im)
pixels.shape

(231, 1274)

#### Padding the image

Pad the image based on kernel size

In [3]:
def check_range(x): return (np.min(x), np.max(x))

In [4]:
check_range(pixels)

(0, 255)

**Some test kernels**

- a = identity
- b = box blur
- c = horizontal derivative
- d = gaussian
- e = sharpening
- f = derivative of gaussian

In [5]:
a = np.zeros((3,3))
a[1,1] = 1
a

array([[0., 0., 0.],
       [0., 1., 0.],
       [0., 0., 0.]])

In [6]:
b = np.ones((3,3))
b

array([[1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.]])

In [7]:
c = np.zeros((3,3))
c[1,0], c[1,2] = -1, 1
c

array([[ 0.,  0.,  0.],
       [-1.,  0.,  1.],
       [ 0.,  0.,  0.]])

In [8]:
d = np.array([[0.003,0.013,0.022,0.013,0.003],
             [0.013,0.059,0.097,0.059,0.013],
             [0.022,0.097,0.159,0.097,0.022],
             [0.013,0.059,0.097,0.059,0.013],
             [0.003,0.013,0.022,0.013,0.003]])
d

array([[0.003, 0.013, 0.022, 0.013, 0.003],
       [0.013, 0.059, 0.097, 0.059, 0.013],
       [0.022, 0.097, 0.159, 0.097, 0.022],
       [0.013, 0.059, 0.097, 0.059, 0.013],
       [0.003, 0.013, 0.022, 0.013, 0.003]])

In [9]:
# vary the value of alpha
e = np.zeros((5,5))
e[1:4,1:4] = a
alpha = 0.8 + 1
e = e * alpha - d
e

array([[-0.003, -0.013, -0.022, -0.013, -0.003],
       [-0.013, -0.059, -0.097, -0.059, -0.013],
       [-0.022, -0.097,  1.641, -0.097, -0.022],
       [-0.013, -0.059, -0.097, -0.059, -0.013],
       [-0.003, -0.013, -0.022, -0.013, -0.003]])

In [10]:
f = np.zeros((5,5))
f[1:4,1:4] = c
f = f * d
f

array([[ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.   , -0.097,  0.   ,  0.097,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ]])

In [11]:
def pad_image(x, kernel):
    """
    Create a new numpy array with all values 255 and then add the image in between
    i/p: image, kernel
    o/p: padded_kernelx
    """
    to_add = kernel.shape[0] - 1
    padded_image = np.full((x.shape[0] + to_add, x.shape[1] + to_add), 255)
    t = to_add // 2
    padded_image[t: padded_image.shape[0] - t, t: padded_image.shape[1] - t] = x
    return padded_image

In [12]:
p = pad_image(pixels, a)
p.shape

(233, 1276)

In [13]:
assert p.shape == (pixels.shape[0] + a.shape[0] - 1, pixels.shape[1] + a.shape[0] - 1)

In [14]:
def display_np(x):
    """
    Display a numpy array as an image
    """
    result = Image.fromarray(x.astype('uint8'))
    result.show()

In [15]:
display_np(p)

#### convolution

In [16]:
def convolve(image, kernel, div = 1):

    # how much subset of the image to take at a time eg 3*3
    size = kernel.shape[0]
    to_remove = kernel.shape[0] - 1

    # initialize a canvas for the output with 0s. We will fill values in this
    output = np.full(image.shape, 255)
    for i in range(image.shape[0] - to_remove):
        for j in range(image.shape[1] - to_remove):
            output[i][j] = np.sum(kernel * image[i:i+size, j:j+size]) / div
    np.clip(output, 0, 255, out = output)

    display_np(output)

In [17]:
convolve(p, a)
convolve(p, b, 9)
convolve(p, c)
convolve(p, d)
convolve(p, e)
convolve(p, f)